# <font color="#418FDE" size="6.5" uppercase>**Eval Exec Compile**</font>

>Last update: 20251221.
    
By the end of this Lecture, you will be able to:
- Describe how eval, exec, and compile execute code strings and code objects within given namespaces. 
- Use eval and exec in controlled examples that restrict available globals and locals to reduce risk. 
- Identify scenarios where these built-ins should be avoided in favor of safer alternatives. 


## **1. Safe eval usage**

### **1.1. Expression Only Evaluation**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_10/Lecture_A/image_01_01.jpg?v=1766298233" width="250">



>* Limit eval to expressions that return values
>* Prevents dangerous actions like file or network access

>* Users customize behavior with value-only expressions
>* Prevents harmful actions while keeping dynamic flexibility

>* Expressions run in namespaces you carefully expose
>* Limit available tools so results stay controlled



In [ ]:
#@title Python Code - Expression Only Evaluation

# Demonstrate eval with expression only evaluation safely.
# Compare safe expression evaluation with unsafe statement execution.
# Show restricted namespace usage for controlled expression evaluation.

# Define a simple safe namespace with math related names only.
safe_globals = {"__builtins__": {}, "miles_to_km": 1.60934}

# User provides a distance expression using only allowed names and numbers.
user_expression = "3 * miles_to_km + 5"

# Evaluate the expression, which only computes a numeric value result.
result = eval(user_expression, safe_globals, {})

# Show the original expression and the computed numeric result clearly.
print("Expression:", user_expression, "=>", "Result:", round(result, 4))

# Now show a dangerous string that tries executing a statement command.
unsafe_code = "__import__('os').system('echo hacked')"

# Using eval with this string would be dangerous and must be avoided.
print("Unsafe expression blocked, not evaluated:", unsafe_code)



### **1.2. Controlled global namespaces**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_10/Lecture_A/image_01_02.jpg?v=1766298276" width="250">



>* Use a custom globals dict as sandbox
>* Expose only safe names; hide powerful functions

>* Small, explicit globals make eval behavior predictable
>* Domain-specific namespaces simplify auditing and debugging later

>* Apps use restricted globals for user expressions
>* Controlled namespaces enable flexibility while enforcing security



In [ ]:
#@title Python Code - Controlled global namespaces

# Demonstrate eval using restricted global namespaces safely.
# Show difference between full globals and limited globals clearly.
# Highlight how missing names stay invisible inside evaluated expressions.

# Define a simple helper function that converts miles to kilometers.
def miles_to_km(miles_value):
    return miles_value * 1.60934

# Build a full globals dictionary that exposes the helper and builtins.
full_globals = {"miles_to_km": miles_to_km, "__builtins__": __builtins__}

# Build a restricted globals dictionary that exposes only the helper function.
restricted_globals = {"miles_to_km": miles_to_km}

# Evaluate an expression using the full globals, allowing builtins like round.
result_full = eval("round(miles_to_km(10), 2)", full_globals)

# Evaluate a similar expression using restricted globals without round available.
try:
    result_restricted = eval("round(miles_to_km(10), 2)", restricted_globals)
except NameError as error:
    result_restricted = f"NameError raised: {error}"  

# Print results to compare behavior between full and restricted namespaces.
print("With full globals, result is:", result_full)

# Show that restricted globals hide round, causing a NameError instead.
print("With restricted globals, behavior is:", result_restricted)



### **1.3. Avoiding untrusted input**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_10/Lecture_A/image_01_03.jpg?v=1766298323" width="250">



>* Never execute code built from outside input
>* Untrusted code can leak data or change behavior

>* String safety checks are weak and bypassable
>* Executing untrusted text blurs data and code boundaries

>* Treat external input strictly as data, never code
>* Use constrained operations or DSLs for flexibility safely



In [ ]:
#@title Python Code - Avoiding untrusted input

# Demonstrate why evaluating untrusted user input is dangerous.
# Show a naive unsafe calculator using eval with raw input.
# Then show a safer alternative that treats input as plain data.

import ast

# Unsafe calculator directly evaluates user expression string.
user_expression = "__import__('os').listdir('.')"  # Simulated untrusted input.
print("Unsafe calculator evaluating expression from untrusted source.")

# This eval call could access files, environment, or worse on real systems.
try:
    unsafe_result = eval(user_expression)
    print("Unsafe eval result shows directory listing:", unsafe_result)
except Exception as error:
    print("Unsafe eval raised error instead of safe rejection:", error)

# Safer calculator only allows numeric literals and arithmetic operators.
print("\nSafer calculator rejecting dangerous non numeric expression.")

try:
    parsed_tree = ast.parse(user_expression, mode="eval")
    allowed_nodes = (ast.Expression, ast.BinOp, ast.UnaryOp, ast.Num, ast.Constant)
    if not isinstance(parsed_tree, allowed_nodes):
        raise ValueError("Expression structure not allowed for safe evaluation.")
    for node in ast.walk(parsed_tree):
        if not isinstance(node, allowed_nodes + (ast.Add, ast.Sub, ast.Mult, ast.Div)):
            raise ValueError("Expression contains forbidden operations or names.")
    safe_result = eval(compile(parsed_tree, "<safe>", "eval"), {"__builtins__": {}}, {})
    print("Safe calculator result using restricted arithmetic only:", safe_result)
except Exception as error:
    print("Safe calculator blocked untrusted expression successfully:", error)



## **2. Safe exec usage**

### **2.1. Running Code Safely**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_10/Lecture_A/image_02_01.jpg?v=1766298365" width="250">



>* Treat all dynamic code as untrusted input
>* Limit its access using sandboxed, restricted environments

>* Define exactly what dynamic code may do
>* Expose only needed capabilities; hide everything else

>* Dynamic code can harm performance and reliability
>* Use monitoring, limits, and isolation to contain failures



### **2.2. Controlling exec namespaces**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_10/Lecture_A/image_02_02.jpg?v=1766298406" width="250">



>* Run dynamic code inside restricted namespaces
>* Expose only chosen functions and data to code

>* Separate exec namespaces contain new variables safely
>* Isolation prevents leaks, conflicts, and lingering state

>* Limit namespace names to control code powers
>* Expose only safe helpers and immutable, read-only data



In [ ]:
#@title Python Code - Controlling exec namespaces

# Demonstrate exec with controlled namespaces safely.
# Show how globals dictionary limits visible names.
# Show that main program variables stay unchanged.

# Define a dangerous looking code string for demonstration.
code_text = "result = secret_value * 2; app_setting = 'changed'"

# Define main program variables that should stay protected.
secret_value = 100
app_setting = "original"

# Create a restricted globals dictionary for executed code.
safe_globals = {"secret_value": secret_value}

# Execute code inside the restricted globals dictionary.
exec(code_text, safe_globals)

# Show what happened inside the restricted globals dictionary.
print("Inside safe_globals result:", safe_globals.get("result"))

# Show that main program variables remain unchanged.
print("Main secret_value still:", secret_value)

# Show that main app_setting variable was not modified.
print("Main app_setting still:", app_setting)



### **2.3. Managing Global Side Effects**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_10/Lecture_A/image_02_03.jpg?v=1766298452" width="250">



>* Dynamic code can secretly change shared globals
>* Treat globals as critical; tightly isolate executed code

>* Run dynamic code in a separate sandbox
>* Expose only safe helpers so changes stay contained

>* Sandboxed code can still affect external systems
>* Expose limited, mostly read-only, carefully audited objects



In [ ]:
#@title Python Code - Managing Global Side Effects

# Show how exec can change global variables dangerously.
# Then show how sandboxed globals prevent unwanted global changes.
# Demonstrate safer exec usage with isolated global side effects.

real_config = {"tax_rate": 0.07, "currency": "USD"}
real_log_messages = []

print("Before unsafe exec, real_config is:", real_config)
print("Before unsafe exec, real_log_messages is:", real_log_messages)

unsafe_code = "tax_rate = 0.25\nreal_config['tax_rate'] = tax_rate\nreal_log_messages.append('Changed tax rate')"
exec(unsafe_code, globals(), locals())

print("After unsafe exec, real_config is:", real_config)
print("After unsafe exec, real_log_messages is:", real_log_messages)

sandbox_globals = {"real_config": {"tax_rate": 0.07, "currency": "USD"}, "real_log_messages": []}
sandbox_code = "tax_rate = 0.30\nreal_config['tax_rate'] = tax_rate\nreal_log_messages.append('Sandbox change')"

exec(sandbox_code, sandbox_globals)

print("After sandbox exec, real_config is still:", real_config)
print("After sandbox exec, real_log_messages is still:", real_log_messages)

print("Sandbox globals now contain:", sandbox_globals["real_config"], sandbox_globals["real_log_messages"])



## **3. Safe use of compile**

### **3.1. Compiling Code Safely**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_10/Lecture_A/image_03_01.jpg?v=1766298495" width="250">



>* Treat all compiled source as untrusted input
>* Avoid compiling user text; prefer safer formats

>* Plan where compiled code runs and accesses
>* Use minimal, restricted namespaces to limit power

>* Validate and restrict code structure before compiling
>* Use limited syntax and namespaces to reduce risk



In [ ]:
#@title Python Code - Compiling Code Safely

# Demonstrate compiling safe expressions using restricted variables only.
# Show how unsafe expressions are rejected before compilation happens.
# Highlight that compiled code runs inside a limited safe namespace.

import ast

# Define allowed node types for very simple arithmetic expressions.
ALLOWED_NODES = (ast.Expression, ast.BinOp, ast.UnaryOp, ast.Num, ast.Name, ast.Load)

# Define allowed operators for basic arithmetic operations only.
ALLOWED_OPS = (ast.Add, ast.Sub, ast.Mult, ast.Div, ast.Pow, ast.USub)


def is_safe_expression(source_text):
    """Check expression structure using abstract syntax tree before compiling."""
    try:
        parsed_tree = ast.parse(source_text, mode="eval")
    except SyntaxError:
        return False

    for node in ast.walk(parsed_tree):
        if not isinstance(node, ALLOWED_NODES) and not isinstance(node, ALLOWED_OPS):
            return False
    return True


# Define a restricted namespace with only numeric variables allowed.
safe_globals = {"__builtins__": {}}

safe_locals = {"miles": 10, "hours": 2}


# Define user provided expressions, including one clearly unsafe attempt.
user_expressions = [
    "miles / hours",
    "miles ** 2 + hours * 3",
    "__import__('os').system('echo hacked')",
]


for expression in user_expressions:
    print(f"Trying expression: {expression}")
    if not is_safe_expression(expression):
        print("Rejected expression, structure not allowed here.")
        print("---")
        continue

    compiled_expr = compile(expression, filename="<user_expr>", mode="eval")

    result = eval(compiled_expr, safe_globals, safe_locals)

    print(f"Safe result value: {result}")
    print("---")




### **3.2. Compile modes overview**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_10/Lecture_A/image_03_02.jpg?v=1766298538" width="250">



>* Compile mode defines construct type and execution
>* Expression mode is limited, safer, value-focused

>* Single-statement mode allows expressions and many statements
>* Dangerous with untrusted input; needs strong isolation

>* File mode compiles full scripts and modules
>* Use only for trusted code, avoid user input



In [ ]:
#@title Python Code - Compile modes overview

# Show how compile modes change allowed code and safety expectations.
# Compare expression mode, single statement mode, and full script mode.
# Highlight why broader modes increase risk with untrusted code.

source_expression = "3 * (5 + 2)"
compiled_expression = compile(source_expression, filename="<expr>", mode="eval")
result_expression = eval(compiled_expression, {"__builtins__": {}}, {})
print("Expression mode result value only:", result_expression)

source_statement = "x = 10\nprint('Single statement mode x value:', x)"
compiled_statement = compile(source_statement, filename="<single>", mode="single")
exec_globals_single = {"__builtins__": {"print": print}}
exec(compiled_statement, exec_globals_single, None)

source_script = "total_inches = 12 * 3\nprint('Script mode total inches:', total_inches)"
compiled_script = compile(source_script, filename="<script>", mode="exec")
exec_globals_script = {"__builtins__": {"print": print}}
exec(compiled_script, exec_globals_script, None)



### **3.3. Reusing Compiled Objects**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Python 3.12 Built-ins A-Z/Module_10/Lecture_A/image_03_03.jpg?v=1766298581" width="250">



>* Compile validated code once under strict controls
>* Reuse vetted code object safely with changing data

>* Separate rule creation from later rule execution
>* Reuse vetted code objects, never recompile user input

>* Treat compilation as a tightly controlled privilege
>* Precompile trusted formulas; never compile live input



In [ ]:
#@title Python Code - Reusing Compiled Objects

# Demonstrate compiling once, then reusing compiled expressions safely.
# Show how different data rows reuse the same compiled transformation.
# Highlight separation between trusted code and changing input values.

# Define a trusted expression string for converting inches to centimeters.
trusted_expression_string = "value_in_inches * 2.54"

# Compile the trusted expression once using eval mode for expressions.
compiled_expression_object = compile(trusted_expression_string, "<trusted_rule>", "eval")

# Example data rows representing lengths in inches for different boards.
board_lengths_inches = [8, 12, 20, 36]

# Process each board length using the same compiled expression object.
for length_in_inches in board_lengths_inches:

    # Prepare local variables dictionary for the current board length.
    local_variables = {"value_in_inches": length_in_inches}

    # Evaluate the precompiled expression with controlled locals and empty globals.
    length_in_centimeters = eval(compiled_expression_object, {}, local_variables)

    # Print a short summary showing original and converted board lengths.
    print(f"Board length {length_in_inches} in equals {length_in_centimeters:.2f} cm.")



# <font color="#418FDE" size="6.5" uppercase>**Eval Exec Compile**</font>


In this lecture, you learned to:
- Describe how eval, exec, and compile execute code strings and code objects within given namespaces. 
- Use eval and exec in controlled examples that restrict available globals and locals to reduce risk. 
- Identify scenarios where these built-ins should be avoided in favor of safer alternatives. 

In the next Lecture (Lecture B), we will go over 'Slices Hash Format'